# Understanding Data Actions

EvalML streamlines the creation and implementation of machine learning models for tabular data. One of the many features it offers is data checks, which are geared towards determining the health of the data before we train a model on it. These data checks have associated actions with them and will be shown in this notebook. In our default data checks, we have the following checks:

- HighlyNullDataCheck: Checks whether the rows or columns are highly null

- IDColumnsDataCheck: Checks for columns that could be ID columns

- TargetLeakageDataCheck: Checks if any of the input features have high association with the associated targets

- InvalidTargetDataCheck: Checks if there are null or other invalid features in the targets

- NoVarianceDataCheck: Checks if any targets or features have no variance

- NaturalLanguageNaNDataCheck: Checks if any natural language columns have missing data

- DateTimeNaNDataCheck: Checks if any datetime columns have missing data


EvalML has additional data checks that can be seen [here](https://evalml.alteryx.com/en/stable/api_index.html#data-checks), with usage examples [here](https://evalml.alteryx.com/en/stable/user_guide/data_checks.html). Below, we will walk through usage of EvalML's default data checks and actions.


First, we import the necessary requirements to demonstrate these checks.

In [ ]:
import woodwork as ww
import pandas as pd
from evalml import AutoMLSearch
from evalml.demos import load_fraud
from evalml.preprocessing import split_data

Let's look at the X data. EvalML uses the [Woodwork](https://woodwork.alteryx.com/en/stable/) library to represent this data. The demo data that EvalML returns is of Woodwork's DataTable and DataColumn types.

In [ ]:
X, y = load_fraud(n_rows=1500)
X

# Adding noise and unclean data

This data is already clean and compatible with EvalML's ``AutoMLSearch``. In order to demonstrate EvalML default data checks, we will add the following:

- A column of mostly null values (<0.5% non-null)

- A column with low/no variance

- A row of null values

- A missing target value


We will add the first two columns to the whole dataset and we will only add the last two to the training data only. Note: these only represent some of the scenarios that EvalML default data checks can catch.

In [ ]:
# add a column with no variance in the data
X['no_variance'] = [1 for _ in range(X.shape[0])]

# add a column with >99.5% null values
X['mostly_nulls'] = [None] * (X.shape[0] - 5) + [i for i in range(5)]

# since we changed the data, let's reinitialize the woodwork datatable
X.ww.init()
# let's split some training and validation data
X_train, X_valid, y_train, y_valid = split_data(X, y, problem_type='binary')

In [ ]:
# let's copy the datetime at row 1 for future use
date = X_train.iloc[1]['datetime']

# make row 1 all nan values
X_train.iloc[1] = [None] * X_train.shape[1]

# make one of the target values null
y_train[990] = None

X_train.ww.init()
y_train = ww.init_series(y_train)
# Let's take another look at the new X_train data
X_train

If we call `AutoMLSearch.search()` on this data, the search will fail due to the columns and issues we've added above. This is because there are a lot of issues with the input data (issues that we added). Note: we use a try/except here to catch the resulting ValueError that AutoMLSearch raises.

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')
try:
    automl.search()
except ValueError as e:
    # to make the error message more distinct
    print("=" * 60, "\n")
    print("=" * 60, "\n")
    print("Search errored out! Message received is: {}".format(e))
    print("=" * 60, "\n")
    print("=" * 60, "\n")

We can use the search function provided in EvalML to determine what potential health issues our data has. We can see that this [search](https://evalml.alteryx.com/en/stable/autoapi/evalml/automl/index.html#evalml.automl.search) function is a public method available through `evalml.automl` and is different from the [search](https://evalml.alteryx.com/en/stable/autoapi/evalml/automl/index.html#evalml.automl.AutoMLSearch) function of the `AutoMLSearch` class in EvalML.

In [ ]:
from evalml.automl import search
results = search(X_train, y_train, problem_type='binary')
results

The return value of the `search` function above is a tuple. The first element is the `AutoMLSearch` object if it runs (and `None` otherwise), and the second element is a dictionary of potential warnings and errors that the default data checks find on the passed-in `X` and `y` data. In this dictionary, warnings are suggestions that the datachecks give that can useful to address to make the search better but will not break AutoMLSearch. On the flip side, errors will break AutoMLSearch and need to be addressed by the user.

## Addressing DataCheck errors
We will show that we can address errors to allow AutoMLSearch to run. However, ignoring warnings will come at the expense of performance.

We can print out the errors first to make it easier to read, and then we'll create new features and targets from the old ones first.

In [ ]:
results[1]['errors']

In [ ]:
# copy the DataTables to new variables
X_train_errors = X_train.copy()
y_train_errors = y_train.copy()

# We address the errors by looking at the resulting dictionary errors listed

# first, let's address the `TARGET_HAS_NULL` error
y_train_errors.fillna(False, inplace=True)

# here, we address the `NO_VARIANCE` error 
X_train_errors.drop("no_variance", axis=1, inplace=True)

# lastly, we address the `DATETIME_HAS_NAN` error with the date we had saved earlier
X_train_errors.iloc[1, 2] = date

# let's reinitialize the Woodwork DataTable
X_train_errors.ww.init()
X_train_errors.head()

We can now run search on `X_train_errors` and `y_train_errors`. Note that the search here doesn't fail since we addressed the errors, but there will still exist warnings in the returned tuple. This search allows the `mostly_nulls` column to remain in the features during search.

In [ ]:
results_errors = search(X_train_errors, y_train_errors, problem_type='binary')
results_errors

## Addressing all warnings and errors
We can look at the `actions` key of the dictionary in order to see what how we can fix and clean all of the data. This will help us clean both the warnings and errors from the data and provide us with a better model.

In [ ]:
results[1]['actions']

We note that there are four action tasks that we can take to clean the data. Three of the tasks ask us to drop a row or column in the features, while one task asks us to impute the target value. 

In [ ]:
# The first action states to drop the row given by the action code
X_train.drop(1477, axis=0, inplace=True)
# we must also drop this for y since we are removing its associated feature input
y_train.drop(index=1477, inplace=True)

print("The new length of X_train is {} and y_train is {}".format(len(X_train),len(y_train)))

In [ ]:
# Remove the 'mostly_nulls' column from X_train, which is the second action item
X_train.drop('mostly_nulls', axis=1, inplace=True)
X_train.head()

In [ ]:
# Address the null in targets, which is the third action item
y_train.fillna(False, inplace=True)
y_train.isna().any()

In [ ]:
# Finally, we can drop the 'no_variance' column, which is the final action item
X_train.drop('no_variance', axis=1, inplace=True)
X_train.head()

In [ ]:
# let's reinitialize the dataframe using Woodwork and try the search again
X_train.ww.init()
results_cleaned = search(X_train, y_train, problem_type='binary')

Note that this time, we do get an `AutoMLSearch` object returned us, as well as an empty dictionary of warnings and errors. We can use the `AutoMLSearch` object as needed, and we can see that the resulting warning dictionary is empty. 

In [ ]:
aml = results_cleaned[0]
aml.rankings

In [ ]:
warnings_dic = results_cleaned[1]
warnings_dic

## Comparing removing only errors versus removing both warnings and errors
Let's see the differences in model performance when we remove only errors versus remove both warnings and errors. To do this, we compare the performance of the best pipelines on the validation data. Remember that in the search where we only address errors, we still have the `mostly_nulls` column present in the data, so we leave that column in the validation data for its respective search. We drop the other `no_variance` column from both searches.

Additionally, we do some logical type setting since we had added additional noise to just the training data. This allows the data to be of the same types in both training and validation.

In [ ]:
# drop the no_variance column
X_valid.drop("no_variance", axis=1, inplace=True)

# logical type management
X_valid.ww.init(logical_types={"customer_present": "Categorical"})
y_valid = ww.init_series(y_valid, logical_type="Categorical")

best_pipeline_errors_only = results_errors[0].best_pipeline
print("Only dropping errors:", best_pipeline_errors_only.score(X_valid, y_valid, ["Log Loss Binary"]), "\n")

# drop the mostly_nulls column and reinitialize the DataTable
X_valid.drop("mostly_nulls", axis=1, inplace=True)
X_valid.ww.init()

best_pipeline_clean = results_cleaned[0].best_pipeline
print("Addressing all actions:", best_pipeline_clean.score(X_valid, y_valid, ["Log Loss Binary"]), "\n")

We see that the performance is slightly better when we address all action items (warnings and errors) in comparison to when we only address errors. While it isn't guaranteed that addressing all actions will always have better performance, we do recommend doing so since we only raise these issues when we believe the features have problems that could negatively impact or not benefit the search.

In the future, we aim to provide a helper function to allow users to quickly clean the data by taking in the list of actions and creating an appropriate pipeline of transformers to alter the data.